## Drive mount/ Common

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
PROJECT_PATH = ROOT+'/My Drive/Fall2020/My_Capstone/'
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "{PROJECT_PATH}"
#!git clone -b Pritam "{GIT_PATH}"
%cd "Edge"
# %%writefile train_california.py
# import importlib
# from utils.post_training_quantization import multipoint_quantization as mt
# importlib.reload(mt)

/content/drive/My Drive/Fall2020/My_Capstone
/content/drive/My Drive/Fall2020/My_Capstone/Edge


In [ ]:
import importlib
from utils.post_training_quantization import multipoint_quantization as mt
importlib.reload(mt)

In [ ]:
# using models written in results csv, find accuracy on train and test data
def quantization_eval_results(results,train_set,test_set,batch_size,criterion):
  train_loss_list = []
  train_accuracy_list = []
  test_loss_list = []
  test_accuracy_list = []
  for i in results["model_artifact"]:
    train_loss, train_accuracy = evaluate(model=i, 
                                        test_set = train_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0) 
    test_loss, test_accuracy = evaluate(model=i, 
                                        test_set = test_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0)  
    train_loss_list.append(train_loss.item())
    test_loss_list.append(test_loss.item())
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
  # append rest of results
  results["train_loss"] = train_loss_list
  results["train_acc"] = train_accuracy_list
  results["test_loss"] = test_loss_list
  results["test_acc"] = test_accuracy_list
  return results

# **Model = ConvNet, Data = CIFAR100 data**
Val Accuracy=46%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN
from utils.post_training_quantization import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_cnn_train.log", "a")
  val_log = open("log/cifar_cnn_val.log", "a")
  test_log = open("log/cifar_cnn_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(train_model, quantize):
  ### config params 
  output_classes = 100
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_cnn_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if train_model:
    model = CNN(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = False)
    torch.save(model, os.path.join(model_dir, model_name))

  else:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  if quantize:
    path_result = "data/results/multipoint/"
    # Choose Quantization method
    results = multipoint_quantization.multipoint_quantization(model_name, precision=[8,6,4,2])
    # results = quantization(model_name, method='all')

    # Evaluate quantized models
    model_results = quantization_eval_results(results,train_set,val_set,batch_size,criterion)
    model_results.to_csv(path_result + model_name[:-3]+'_multipoint2' +".csv")
    print(model_results)


if __name__ == "__main__":
  main(train_model=False, quantize=True)


Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 2.022034, validation accuracy = 0.460300

--------Quantizing the model cifar_cnn_model.pt with precision 2
All layers except bias layers:  ['network.0.weight', 'network.1.weight', 'network.3.weight', 'network.7.weight', 'network.8.weight', 'network.11.weight', 'network.12.weight', 'network.15.weight', 'network.16.weight', 'network.18.weight', 'network.19.weight', 'network.24.weight', 'network.26.weight']

Quantizing layer:network.1.weight , with weights shape:torch.Size([32])
Final error of W quantization: 5.127986431121826

Quantizing layer:network.3.weight , with weights shape:torch.Size([64, 32, 3, 3])
Final error of W quantization: 23.36138916015625

Quantizing layer:network.7.weight , with weights shape:torch.Size([128, 64, 3, 3])
Final error of W quantization: 47.049110412597656

Quantizing layer:network.8.weight , with weights shape:torch.Size([128])
Final error of W quantization: 11.08896541595459

Quantizing layer:network.


                model quant_method precision  ... train_acc  test_loss  test_acc
0  cifar_cnn_model.pt  multi-point        32  ...   0.58748   2.022032    0.4603
1  cifar_cnn_model.pt  multi-point         2  ...   0.01000   4.627048    0.0100

[2 rows x 8 columns]


# **Model = Resnet9, Data = CIFAR100 data**
Val Accuracy=60%



In [ ]:
# %%writefile train_cifar.py
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet9
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.post_training_quantization import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-5

def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_resnet9_train.log", "a")
  val_log = open("log/cifar_resnet9_val.log", "a")
  test_log = open("log/cifar_resnet9_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)

  # trainloader
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=max_lr, weight_decay=weight_decay)
  sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      nn.utils.clip_grad_value_(model.parameters(), grad_clip)
      optimizer.step()
      sched.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(train_model, quantize):
  ### config params
  output_classes = 100
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_resnet9_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)
  
  if train_model:
    model = CNN_resnet9(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))
  else:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))


  if quantize:
    path_result = "data/results/multipoint/"
    # Choose Quantization method
    results = multipoint_quantization.multipoint_quantization(model_name, precision=[2])
    # results = quantization(model_name, method='all')

    # Evaluate quantized models
    model_results = quantization_eval_results(results,train_set,val_set,batch_size,criterion)
    model_results.to_csv(path_result + model_name[:-3]+'_multipoint2' +".csv")
    print(model_results)


if __name__ == "__main__":
  main(train_model=False, quantize=True)

Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 1.599502, validation accuracy = 0.597900

--------Quantizing the model cifar_resnet9_model.pt with precision 2
All layers except bias layers:  ['conv1.0.weight', 'conv1.1.weight', 'conv2.0.weight', 'conv2.1.weight', 'res1.0.0.weight', 'res1.0.1.weight', 'res1.1.0.weight', 'res1.1.1.weight', 'conv3.0.weight', 'conv3.1.weight', 'conv4.0.weight', 'conv4.1.weight', 'res2.0.0.weight', 'res2.0.1.weight', 'res2.1.0.weight', 'res2.1.1.weight', 'classifier.2.weight']

Quantizing layer:conv1.1.weight , with weights shape:torch.Size([64])
Final error of W quantization: 9.3154935836792

Quantizing layer:conv2.0.weight , with weights shape:torch.Size([128, 64, 3, 3])
Final error of W quantization: 65.13798522949219

Quantizing layer:conv2.1.weight , with weights shape:torch.Size([128])
Final error of W quantization: 15.2453031539917

Quantizing layer:res1.0.0.weight , with weights shape:torch.Size([128, 128, 3, 3])
Final error of W quantization


                    model quant_method  ... test_loss test_acc
0  cifar_resnet9_model.pt  multi-point  ...  1.599501   0.5979
1  cifar_resnet9_model.pt  multi-point  ...  5.075922   0.0100

[2 rows x 8 columns]


# **Model = Resnet50, Data = CIFAR100 data**
Val Accuracy=50%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet50
from utils.post_training_quantization import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_resnet50_train.log", "a")
  val_log = open("log/cifar_resnet50_val.log", "a")
  test_log = open("log/cifar_resnet50_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(train_model, quantize):
  ### config params
  output_classes = 100
  learning_rate = 0.0009
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_resnet50_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if train_model:
    model = CNN_resnet50(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))
  else:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))


  if quantize:
    path_result = "data/results/multipoint/"
    # Choose Quantization method
    results = multipoint_quantization.multipoint_quantization(model_name, precision=[8,6,4,2])
    # results = quantization(model_name, method='all')

    # Evaluate quantized models
    model_results = quantization_eval_results(results,train_set,val_set,batch_size,criterion)
    model_results.to_csv(path_result + model_name[:-3]+'_multipoint2' +".csv")
    print(model_results)


if __name__ == "__main__":
  main(train_model=False, quantize=True)

Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 1.780087, validation accuracy = 0.502900
Files already downloaded and verified

--------Quantizing the model cifar_resnet50_model.pt with precision 8
All layers:  ['network.conv1.weight', 'network.bn1.weight', 'network.layer1.0.conv1.weight', 'network.layer1.0.bn1.weight', 'network.layer1.0.conv2.weight', 'network.layer1.0.bn2.weight', 'network.layer1.0.conv3.weight', 'network.layer1.0.bn3.weight', 'network.layer1.0.downsample.0.weight', 'network.layer1.0.downsample.1.weight', 'network.layer1.1.conv1.weight', 'network.layer1.1.bn1.weight', 'network.layer1.1.conv2.weight', 'network.layer1.1.bn2.weight', 'network.layer1.1.conv3.weight', 'network.layer1.1.bn3.weight', 'network.layer1.2.conv1.weight', 'network.layer1.2.bn1.weight', 'network.layer1.2.conv2.weight', 'network.layer1.2.bn2.weight', 'network.layer1.2.conv3.weight', 'network.layer1.2.bn3.weight', 'network.layer2.0.conv1.weight', 'network.layer2.0.bn1.weight', 'network.layer2


Train Accuracy:  0.54642  Test Accuracy:  0.5029



Train Accuracy:  0.553  Test Accuracy:  0.5137



Train Accuracy:  0.01  Test Accuracy:  0.01



Train Accuracy:  0.01  Test Accuracy:  0.01



Train Accuracy:  0.01  Test Accuracy:  0.01
                     model quant_method  ... test_loss test_acc
0  cifar_resnet50_model.pt  multi-point  ...  1.780089   0.5029
1  cifar_resnet50_model.pt  multi-point  ...  1.740885   0.5137
2  cifar_resnet50_model.pt  multi-point  ...  5.101506   0.0100
3  cifar_resnet50_model.pt  multi-point  ...  4.611042   0.0100
4  cifar_resnet50_model.pt  multi-point  ...  4.610587   0.0100

[5 rows x 8 columns]


# **Model = VGG16, Data= CIFAR100**
Val Accuracy=19%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_vgg16
from utils.post_training_quantization import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_vgg16_train.log", "a")
  val_log = open("log/cifar_vgg16_val.log", "a")
  test_log = open("log/cifar_vgg16_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9, nesterov=True)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(train_model, quantize):
  ### config params
  output_classes = 100
  learning_rate = 0.01
  batch_size = 32
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_vgg16_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if train_model:
    model = CNN_vgg16(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))
  else:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))


  if quantize:
    path_result = "data/results/multipoint/"
    # Choose Quantization method
    results = multipoint_quantization.multipoint_quantization(model_name, precision=[2])
    # results = quantization(model_name, method='all')

    # Evaluate quantized models
    model_results = quantization_eval_results(results,train_set,val_set,batch_size,criterion)
    model_results.to_csv(path_result + model_name[:-3]+'_multipoint2' +".csv")
    print(model_results)


if __name__ == "__main__":
  main(train_model=False, quantize=True)

Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 4.429444, validation accuracy = 0.193200

--------Quantizing the model cifar_vgg16_model.pt with precision 2
All layers except bias layers:  ['network.features.0.weight', 'network.features.2.weight', 'network.features.5.weight', 'network.features.7.weight', 'network.features.10.weight', 'network.features.12.weight', 'network.features.14.weight', 'network.features.17.weight', 'network.features.19.weight', 'network.features.21.weight', 'network.features.24.weight', 'network.features.26.weight', 'network.features.28.weight', 'network.classifier.1.weight', 'network.classifier.4.weight', 'network.classifier.6.weight']

Quantizing layer:network.features.2.weight , with weights shape:torch.Size([64, 64, 3, 3])
Final error of W quantization: 9.936873435974121

Quantizing layer:network.features.5.weight , with weights shape:torch.Size([128, 64, 3, 3])
Final error of W quantization: 12.469865798950195

Quantizing layer:network.features.7.wei


                  model quant_method precision  ... train_acc  test_loss  test_acc
0  cifar_vgg16_model.pt  multi-point        32  ...    0.2143   4.429543    0.1932
1  cifar_vgg16_model.pt  multi-point         2  ...    0.0100   4.605170    0.0100

[2 rows x 8 columns]
